# Using Superwise's Basic LLM Agent for Intermediate Language Generation

In [ ]:
!pip install superwise-api --quiet

### Configure Superwise Connection

In [ ]:
import os
from superwise_api.superwise_client import SuperwiseClient

os.environ['SUPERWISE_CLIENT_ID'] = ''
os.environ['SUPERWISE_CLIENT_SECRET'] = ''

sw = SuperwiseClient()    


## Create Superwise Application

First let's select an LLM to use with Superwise. Here we use GPT-3.5-Turbo. When converting from natural language to a simple intermediate language, we do not need a very powerful LLM. Even smaller models like the 3 billion paramater Llama model work.

In [ ]:
from superwise_api.models.application.application import OpenAIModel, OpenAIModelVersion

llm_model = OpenAIModel(version=OpenAIModelVersion.GPT_3_5_TURBO, api_token="Open API token")

Next we need to build the prompt for our intermediate language. 

### Define the Schema

In this notebook, we use json to format our intermediate language and define the schema as:
```
{
    “customer_name”: <customer_name>,
    “request”: <request_type>,
    “start_date”: <start_date>,
    “end_date”: <end_date>
}
```
where request type is exclusive to "customer" or "transactions".

### Build the Prompt

When we build the prompt, we indicate to the LLM that its task is to translate between Natural Language and our intermediate language, provide it with the definition of the intermediate language, and then provide it a few examples of translation.

It is always helpful for the LLM to provide a few example inputs and their output in the intermediate language. The few-shot training does not need to be a robust defintion of every possible question type but should be enough to cover all the bases of the language so the LLM can infer context for the .

In [ ]:
# Change this prompt to use your own Intermediate Language. 
# Few-shot training at the end of the prompt is recommended for best results.

prompt = """Act as a natural language interpreter. Below is a a JSON Schema. Your response should always be in this schema.

Schema:
{{
    “customer_name”: <customer_name>,
    “request”: <request_type>,
    “start_date”: <start_date>,
    “end_date”: <end_date>
}}

The allowed values for request are "customer" or "transactions". The start date and end dates are optional but should always be in "YYYY-MM-DD" format.

Here are some examples of natural language inputs and their schema output.
If prompted with "Details for Box Company" respond:
{{
    “customer_name”: "Box Company",
    “request”: "customer"
}}
If prompted with "2024 transactions for Store LLC" respond:
{{
    “customer_name”: "Store LLC",
    “request”: "transactions",
    “start_date”: "2024-01-01",
    “end_date”: "2024-12-31"
}}
If prompted with "Transactions for Job Corporation since June 2022" respond:
{{
    “customer_name”: "Store LLC",
    “request”: "transactions",
    “start_date”: "2022-06-01"
}}
"""

In [ ]:
from superwise_api.models.application.application import BasicLLMConfig

app = sw.application.create(
    name="NL-to-DSL",
    additional_config=BasicLLMConfig(),
    llm_model=llm_model,
  	prompt=prompt
)
app_id = app.id

# Test Application

In [4]:
def ask_swe_playground(superwise_client: SuperwiseClient, application_id, user_input):
    app = superwise_client.application.get_by_id(application_id)

    application_response = superwise_client.application.ask_playground(input=user_input,
                                                        llm_model=app.llm_model,
                                                        prompt=app.prompt,
                                                        additional_config=app.additional_config,
                                                        chat_history=[])

    return application_response.output

In [ ]:
# Let's create some prompts and test out our application
user_inputs = ["Details for Superwise", 
               "2020 transactions for Suits and Things LLC",
               "Sales for Bag Company in August of 2023",
               "Details for Tile LLC through the end of 2022"]
for user_input in user_inputs:
    print(f"User Input: {user_input}")
    sw_response = ask_swe_playground(superwise_client=sw, application_id=app_id, user_input=user_input)
    print(sw_response)
    print("---------------------------------------------------")

User Input: Details for Superwise
{
    “customer_name”: "Superwise",
    “request”: "customer"
}
---------------------------------------------------
User Input: 2020 transactions for Suits and Things LLC
{
    “customer_name”: "Suits and Things LLC",
    “request”: "transactions",
    “start_date”: "2020-01-01",
    “end_date”: "2020-12-31"
}
---------------------------------------------------
User Input: Sales for Bag Company in August of 2023
{
    “customer_name”: "Bag Company",
    “request”: "transactions",
    “start_date”: "2023-08-01",
    “end_date”: "2023-08-31"
}
---------------------------------------------------
User Input: Details for Tile LLC through the end of 2022
{
    “customer_name”: "Tile LLC",
    “request”: "customer",
    “end_date”: "2022-12-31"
}
---------------------------------------------------


Testing our application, the superwise application leveraging GPT-3.5-Turbo was able to accurately infer the schema. It not only inferred the example schema provided by the few-shot training but was also able to extend that few-shot training to examples available within the defined intermediate language but outside of any of the shots provided in the prompt!